In [141]:
import twint
import numpy as np
import pandas as pd
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [142]:
df = pd.read_csv("../data/raw/TweetsRaw.csv", error_bad_lines=False,lineterminator='\n')

/Users/avinaashpadman/opt/anaconda3/envs/cardanotwittersentiment/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version.





The twitter data was scrapped using a library called snscraper. The script I wrote for that is in the src file as scrapeTweets.py. The API allowed me to scrape 200000 tweets from 2021-08-31 to 2021-08-08 which I then saved in my raw data file as TweetsRaw. It scrapped tweets that contained the word "cardano" in it.  

In [143]:
df.tail()

,Unnamed: 0,Datetime,Tweet ID,Text,User
199996,199996,2021-08-08 22:29:41+00:00,1424498074865504257,@Fougias_ @elliotrades @cryptoknitties @AdaDolls @CardinosNFT @CardanoKidz @spacebudzNFT @CardanoBits @the_hoskinsons @CardanoMonsters @KingsCarda...,ConservationAR
199997,199997,2021-08-08 22:29:40+00:00,1424498068418826243,"How Cardano can help in Ethiopia, El Salvador and other developing countries https://t.co/rjAgSRQfB0",CryptoBerts
199998,199998,2021-08-08 22:29:39+00:00,1424498066137264136,"@eVIRALToken has been changing the game since inception and will continue to do so. Say goodbye to #rugpulls #cryptomarket, any project launched o...",kloflin86
199999,199999,2021-08-08 22:29:27+00:00,1424498014660481026,$ADA 24 Hours Stats #Cardano\n\n24h High: $1.49131\n24h Average: $1.45527\n24h Low: $1.40946,Altcoin_Track
200000,200000,2021-08-08 22:28:13+00:00,1424497705410342918,@project_takumi @cardano_bull @JonMunitz I’m talking about how they keep calling themselves progressives and now they refer it to the people who p...,ForYang4


In [144]:
df.shape

(200001, 5)

In [145]:
df = df.drop_duplicates(subset = ["Text"])

In [146]:
df.shape

(195620, 5)

First thing that I decided to do was to make sure that any duplicate tweets are removed so that the data reflects the sentiment value of each unique tweet. This removes around 4500 duplicated tweets

In [147]:
import re
import nltk 
import string
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

First we have to process the tweets into words that can be more easily analyzed by the sentiment analyzer. I will use re to process the text. We want to remove any @username words and retweets. Using the re.sub function, we can substitute a certain pattern with a blank text, effectively removing the pattern from the text.

In [148]:
df.sample(10)

,Unnamed: 0,Datetime,Tweet ID,Text,User
177365,177365,2021-08-12 14:45:50+00:00,1425830892463816711,"@cryptodoc99 has named this 'Warped Matrix'!! I am thinking of what sort of ""specials"" I could create... welcome any and all ideas #Cardano https:...",marshallchow
127006,127006,2021-08-18 21:37:35+00:00,1428108841904971778,"#ARM1 is a single pool running in Canada, Ontario, property powered by solar panels for more than 3 years.\n\n#Cardano #ADA #solarpower @adapools_...",ArmADApool
85127,85127,2021-08-23 04:54:58+00:00,1429668466198421510,#cardano is fucking unstoppable.,dhruvy0
16314,16314,2021-08-30 07:25:36+00:00,1432243086009122821,When will $ADA.X reach $5 ? #ada #cardano #CardanoADA #CardanoCommunity #CardanoSummit2021 #Cardano360 #CardanoCity #finance #fintech #blockchain ...,Fairmarket_Ph
87383,87383,2021-08-23 01:18:02+00:00,1429613872986603526,It's happening now!!\n🔥🔥$3 #Cardano (#ADA) coming sooner than you think!!\n\n✅ADA = $2.79 https://t.co/pcL4ZuL0xJ https://t.co/Vq5hoiQpo7,Dr_shwetaPHD
117327,117327,2021-08-19 23:36:59+00:00,1428501276120203268,"@AdvocateAda @cardano_whale I don't recall saying pro traders were short, or betting on the downside. even if was a neutral to bearish analysis, i...",noshitcoins
179383,179383,2021-08-12 06:07:54+00:00,1425700553531744257,"Gonna say it again from just a macro level observation, Hedera Hashgraph is massively undervalued $ underperforming at this point, but I truly bel...",chalres200
109100,109100,2021-08-20 14:02:51+00:00,1428719178035474437,"¿Recuerdan en el 2016 cuando decían por LTC que era el nuevo ""BTC""?.\nAhora hablan de que Cardano será el nuevo ETH.\nLa innovación son los camino...",turrom13
117772,117772,2021-08-19 22:45:32+00:00,1428488329042010114,I have decided to retire the pool after 7 months of hard work to concentrate on other dev work. Good luck to my fellow small SPO's. I wish you all...,gerstaking
135526,135526,2021-08-17 14:18:05+00:00,1427635850372861964,@ceterispar1bus @cmsholdings I wish someone hate FTT like cardano,Shershaaah


In [149]:
remove_rt = lambda x: re.sub( 'RT @\w+: ',"",x)
rt = lambda x: re.sub("(@\w+)|(#\w+)|(https:\/\/.*)","",x)
df["ModText"] = df.Text.map(remove_rt)
df["ModText"] = df.Text.map(rt)
df["ModText"] = df["ModText"].str.lower()

Using the regex library I can clean up the text by identifying certain patterns and replacing it with a blank space. For example I can remove the patterns "RT @michael" by using re.sub('RT @\w+:' , "", x). \w+ is a regex command that matches alphanumeric character (\w) of one or more occurences (+). Similarly "#\w+' will match to any hashtag pattern that starts with # and is followed up by an alphanumeric sequence of length one or more.


In [150]:
df.head(10)

,Unnamed: 0,Datetime,Tweet ID,Text,User,ModText
0,0,2021-08-31 23:59:57+00:00,1432855712657412096,@sereingives @nft_sol @nfts_cardano Done,yedamdashing1,done
1,1,2021-08-31 23:59:49+00:00,1432855676729106436,"@jeremeybivens This is a way to store your keys for any type of storage. Yea, you can stake from hot and cold storage on Cardano!",TheADAApe,"this is a way to store your keys for any type of storage. yea, you can stake from hot and cold storage on cardano!"
2,2,2021-08-31 23:59:47+00:00,1432855671343656963,What’s great about NFT’s on Cardano? They’re fun! #cryptodino @cryptodino_io #NFTCommunity #NFT #ADA #ADAgang #cardano #CardanoNFT https://t.co/Yd...,LarryMiles,what’s great about nft’s on cardano? they’re fun!
3,3,2021-08-31 23:59:35+00:00,1432855621192364032,Hoskinson: These are the circumstances in which Cardano $ADA and Solana $SOL will collaborate,CyrilXbt,hoskinson: these are the circumstances in which cardano $ada and solana $sol will collaborate
4,4,2021-08-31 23:58:53+00:00,1432855443617902595,@Cardano_Robin @psychADAlias @Champagnemami02 @tr3090 @w909924,j20702050,
5,5,2021-08-31 23:58:35+00:00,1432855366841344006,Hoskinson: These are the circumstances in which Cardano (ADA) and Solana (SOL) will collaborate #Cryptocurrency,SuddenlySatoshi,hoskinson: these are the circumstances in which cardano (ada) and solana (sol) will collaborate
6,6,2021-08-31 23:58:23+00:00,1432855318820790274,@falko15543940 @golden_doghouse @ValueCryptos @ercwl Note over 100 ETH projects have already stated they will migrate to Cardano. When the UX is t...,WiseVesting101,"note over 100 eth projects have already stated they will migrate to cardano. when the ux is tried and proven, the masses will follow. people a..."
7,7,2021-08-31 23:57:50+00:00,1432855179875991555,"Rob $tone, two damn phones\nBabylons can't crack the code (no)👉👈😁\n#Bitcoin #Solana #XRP #Cardano #Algorand",execut333,"rob $tone, two damn phones\nbabylons can't crack the code (no)👉👈😁\n"
8,8,2021-08-31 23:57:21+00:00,1432855056982945800,"It’s a shame that @SmaugPool isn’t fully saturated. Ticker: SMAUG.\n\nResponsible for https://t.co/BWLvKB6Z7p, a tool that almost everyone in the...",TimothyAyers0,it’s a shame that isn’t fully saturated. ticker: smaug.\n\nresponsible for \n\n
9,9,2021-08-31 23:57:10+00:00,1432855010229039104,Bitcoin Ethereum Cardano polkadot Matic Bullish or Bearish?? What do the... https://t.co/HhpVrlBW73 via @YouTube,MoonDogTrading1,bitcoin ethereum cardano polkadot matic bullish or bearish?? what do the...


In [151]:
df.tail()

,Unnamed: 0,Datetime,Tweet ID,Text,User,ModText
199996,199996,2021-08-08 22:29:41+00:00,1424498074865504257,@Fougias_ @elliotrades @cryptoknitties @AdaDolls @CardinosNFT @CardanoKidz @spacebudzNFT @CardanoBits @the_hoskinsons @CardanoMonsters @KingsCarda...,ConservationAR,nice one. i will look into these! what is the marketplace for cardano?
199997,199997,2021-08-08 22:29:40+00:00,1424498068418826243,"How Cardano can help in Ethiopia, El Salvador and other developing countries https://t.co/rjAgSRQfB0",CryptoBerts,"how cardano can help in ethiopia, el salvador and other developing countries"
199998,199998,2021-08-08 22:29:39+00:00,1424498066137264136,"@eVIRALToken has been changing the game since inception and will continue to do so. Say goodbye to #rugpulls #cryptomarket, any project launched o...",kloflin86,"has been changing the game since inception and will continue to do so. say goodbye to , any project launched on the platform is a safe project...."
199999,199999,2021-08-08 22:29:27+00:00,1424498014660481026,$ADA 24 Hours Stats #Cardano\n\n24h High: $1.49131\n24h Average: $1.45527\n24h Low: $1.40946,Altcoin_Track,$ada 24 hours stats \n\n24h high: $1.49131\n24h average: $1.45527\n24h low: $1.40946
200000,200000,2021-08-08 22:28:13+00:00,1424497705410342918,@project_takumi @cardano_bull @JonMunitz I’m talking about how they keep calling themselves progressives and now they refer it to the people who p...,ForYang4,i’m talking about how they keep calling themselves progressives and now they refer it to the people who praise the squad


In [152]:
positive = 0
negative = 0
neutral= 0
polarity = 0
SentimentScore = []


In [153]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/avinaashpadman/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
for idx, tweet in df.iterrows():
    #print(tweet.Text)
    analysis = TextBlob(tweet.ModText)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.ModText)
    neg = score['neg']
    pos = score['pos']
    neu = score['neg']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    
    if neg > pos:
        SentimentScore.append(-1)
    
    if pos > neg:
        SentimentScore.append(1)
    
    if pos == neg:
        SentimentScore.append(0)
        
        

In [ ]:
df.head()

In [ ]:
df["sentimentScore"] = SentimentScore

In [ ]:
df["sentimentScore"].value_counts()

In [ ]:
df["Datetime"] = pd.to_datetime(df["Datetime"])

In [ ]:
df.dtypes

In [ ]:
df.groupby([pd.Grouper(key='Datetime', freq='D'),"sentimentScore"]).count()

In [ ]:
 DailyTweetSentiment =  df.groupby(pd.Grouper(key='Datetime', freq='D'))["sentimentScore"].sum()

In [ ]:
 DailyTweetSentiment = pd.DataFrame(DailyTweetSentiment)

In [ ]:
DailyTweetSentiment.head()

The DailyTweetSentiment dataframe contains the cumulative sentimentScore for each day since the 8th of August to the 31st of August

In [ ]:
import plotly.express as px

In [ ]:
px.bar(DailyTweetSentiment)

In [ ]:
DailyTweetSentimentByScore = df.groupby([pd.Grouper(key='Datetime', freq='D'),"sentimentScore"])["User"].count()

In [ ]:
DailyTweetSentimentByScore.head()

In [ ]:
DailyTweetSentimentByScore = DailyTweetSentimentByScore.reset_index(level = ["sentimentScore"])

In [ ]:
px.bar(DailyTweetSentimentByScore, color = "sentimentScore")

In [ ]:
PosSentDailyTweets = DailyTweetSentimentByScore[DailyTweetSentimentByScore["sentimentScore"] == 1]
PosSentDailyTweets = PosSentDailyTweets.drop("sentimentScore", axis = 1)

In [ ]:
px.bar(PosSentDailyTweets)

In [ ]:
PosSentDailyTweets.head()

In [ ]:
ADAPrices = pd.read_csv("../data/raw/ADAPrices.csv", names = ['unix', 'date', 'symbol', 'open','high','low','close','volumeADA','volumeUSDT','tradecount'] )

In [ ]:
ADAPrices.head()

I downloaded a csv file of ADA prices from the last three months. After converting it to a Pandas dataframe I had clean up the dataframe by assigning column names to the dataframe that were in the first row and then removing the first two rows. I then converted the date column into a datetime variable type and set it as the index in order to make it easier for me to manipulate and choose the dates that matched the dates I had twitter sentiment for. Finally I reversed the order of the dataframe rows so it matched the order for the twitter sentiment dataframe

In [ ]:
ADAPrices = ADAPrices.drop([0,1])

In [ ]:
ADAPrices["date"] = pd.to_datetime(ADAPrices['date'])

In [ ]:
ADAPrices = ADAPrices.set_index(['date'])

In [ ]:
ADAPricesDated = ADAPrices.loc['2021-08-08' : '2021-08-31']

In [ ]:
ADAPricesDated = ADAPricesDated.iloc[::-1]

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar( y = PosSentDailyTweets["User"], name = "Twitter ADA Sentiment"),
    secondary_y=False,
    )

fig.add_trace(
    go.Scatter(y = ADAPricesDated['close'], name = "ADA Close Price"),
    secondary_y=True,
    )

fig.show()

On face values there is an increasing trend in positive sentiment from August 8 to August 31 and the price trend of ADA shows it increasing at an almost linear rate. Its hard to tell whether the positive sentiment mirrors the price increase mainly because more data is needed to show the sentiment from earlier dates. I have a range from August 8 to August 31 mainly because it took a long time to scrape the 200,000 tweets that mention cardano. However that was a slower process because I used a python script which due to its compiled nature would be slower. In the future I am going to experiments with ways to scrape twitter data at a faster rate. However for the sake of honing my techniques in NLP and data visualization and manipulation, this project gave me a lot of insight.